# 법정동2VEC


### 프로젝트 목표
* 전국 지역 단위별 embedding값을 제공하고자 함

### 사용 가능한 곳
* 전염병 모델에 embedding값을 covariate으로 활용
* 장소 추천/ 예측 모델에 embedding값을 활용

### 데이터
* 전국 사업체조사 공공데이터: https://www.data.go.kr/dataset/3073433/fileData.do
* 전국 건물 DB 공공데이터: http://www.juso.go.kr/addrlink/addressBuildDevNew.do?menu=rdnm

### 제반 지식
* Poincare embedding
    - Gensim implementation: https://radimrehurek.com/gensim/models/poincare.html
    - Pytorch implementation: https://github.com/facebookresearch/poincare-embeddings
* PNU코드: https://m.post.naver.com/viewer/postView.nhn?volumeNo=7135987&memberNo=36248235
    - 예시 ![PNU코드](fig/pnu.jpg)
    - 행안부: 실제 PNU 값은 자주 변경되며 행안부의 주민등록주소코드 자료실을 통해 다운로드 받으실 수 있습니다.
* 건물관리번호 (https://www.vw-lab.com/32):
    - 생성 당시 기준의 PNU + 연번으로 구성되었으며 이후 행정구역이 변동되어도 관리번호는 반영이 안됨
    - 변경되지 않는 고유값
    - 예시 ![건물관리번호](fig/bldgcode.png)
* 기타 지식
    * 주소 변환: http://juso.go.kr/dn.do?fileName=%EC%A3%BC%EC%86%8C%EC%A0%84%ED%99%98_%EA%B0%80%EC%9D%B4%EB%93%9C.pdf&realFileName=f493270c-d88f-4852-a807-17a6189a8871.pdf&regYmd=2012
    * PNU ↔ 주소간 상호 변환과 메타데이터: https://m.post.naver.com/viewer/postView.nhn?volumeNo=7242031&memberNo=36248235

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 39)
pd.set_option('display.float_format', '{:.0f}'.format)
pd.set_option('display.max_rows', 50)

In [2]:
df1 = pd.read_csv('./상가업소정보_201912_01.csv', sep='|')

In [3]:
df2 = pd.read_csv('./상가업소정보_201912_02.csv', sep='|')

/home/sundong/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df3 = pd.read_csv('./상가업소정보_201912_03.csv', sep='|')

/home/sundong/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df4 = pd.read_csv('./상가업소정보_201912_04.csv', sep='|')

In [6]:
df = pd.concat([df1, df2, df3, df4])

In [47]:
df.shape

(2741196, 47)

In [48]:
df.head(5)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,...,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,경도_2,경도_1,경도_0,위도_2,위도_1,위도_0,법정동명전체,시군구명전체
0,19905471,와라와라호프,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11680,강남구,1168060000,대치1동,1168010600,대치동,...,nan,1168010600106040000014378,대치상가,서울특별시 강남구 남부순환로 2933,135280,6280,NaN,1,NaN,127,37,127,127,127,37,38,37,서울특별시 강남구 대치동,서울특별시 강남구
1,19911397,커피빈코리아선릉로93길점,코리아선릉로93길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,11,서울특별시,11680,강남구,1168064000,역삼1동,1168010100,역삼동,...,nan,1168010100106960042022041,NaN,서울특별시 강남구 선릉로93길 6,135080,6149,NaN,1,NaN,127,38,127,127,127,38,38,38,서울특별시 강남구 역삼동,서울특별시 강남구
2,19911801,프로포즈,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11545,금천구,1154563000,독산3동,1154510200,독산동,...,6,1154510200101620001017748,NaN,서울특별시 금천구 가산로 34-6,153010,8545,NaN,1,NaN,127,37,127,127,127,37,38,37,서울특별시 금천구 독산동,서울특별시 금천구
3,19912201,싱싱커피&토스트,NaN,Q,음식,Q07,패스트푸드,Q07A10,토스트전문,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",26,부산광역시,26530,사상구,2653062000,괘법동,2653010400,괘법동,...,nan,2653010400105780000002037,산업용품유통상가,부산광역시 사상구 괘감로 37,617726,46977,NaN,1,26,129,35,129,129,129,35,35,35,부산광역시 사상구 괘법동,부산광역시 사상구
4,19932756,가락사우나내스낵,NaN,F,생활서비스,F09,대중목욕탕/휴게,F09A02,사우나/증기탕/온천,S96121,욕탕업,11,서울특별시,11710,송파구,1171060000,석촌동,1171010500,석촌동,...,nan,1171010500102560005010490,NaN,서울특별시 송파구 가락로 71,138846,5690,NaN,1,NaN,127,38,127,127,127,38,38,38,서울특별시 송파구 석촌동,서울특별시 송파구


In [49]:
for col in df.columns:
    print(f'# of {col}: {df[col].nunique()}, # of null: {df[col].isnull().sum()}')

# of 상가업소번호: 2741196, # of null: 0
# of 상호명: 1482860, # of null: 2
# of 지점명: 107358, # of null: 2407872
# of 상권업종대분류코드: 9, # of null: 0
# of 상권업종대분류명: 9, # of null: 0
# of 상권업종중분류코드: 94, # of null: 0
# of 상권업종중분류명: 94, # of null: 0
# of 상권업종소분류코드: 737, # of null: 0
# of 상권업종소분류명: 737, # of null: 0
# of 표준산업분류코드: 213, # of null: 171325
# of 표준산업분류명: 213, # of null: 171325
# of 시도코드: 17, # of null: 0
# of 시도명: 17, # of null: 0
# of 시군구코드: 251, # of null: 0
# of 시군구명: 228, # of null: 0
# of 행정동코드: 3587, # of null: 0
# of 행정동명: 3198, # of null: 0
# of 법정동코드: 16818, # of null: 7
# of 법정동명: 3920, # of null: 7
# of 지번코드: 1038655, # of null: 0
# of 대지구분코드: 2, # of null: 0
# of 대지구분명: 2, # of null: 0
# of 지번본번지: 5427, # of null: 0
# of 지번부번지: 1582, # of null: 591681
# of 지번주소: 987607, # of null: 0
# of 도로명코드: 107596, # of null: 0
# of 도로명: 110722, # of null: 0
# of 건물본번지: 6061, # of null: 0
# of 건물부번지: 319, # of null: 2233705
# of 건물관리번호: 995570, # of null: 0
# of 건물명: 164632, # of null: 165142

In [50]:
for col in df.columns:
#     print(f'# of {col}: {df[col].nunique()}, # of null: {df[col].isnull().sum()}')    
    print(df[col].value_counts().head(10))


15557580    1
24118759    1
24049524    1
15956509    1
24037234    1
28237681    1
23996270    1
27455037    1
28182378    1
16376128    1
Name: 상가업소번호, dtype: int64
CU                 10074
점프셈교실              10069
GS25                8450
간단하지만특별한화피디Cafe     7311
세븐일레븐               6670
미니스톱                2096
파리바게뜨               2057
크린토피아               1792
이마트24               1785
이디야커피               1760
Name: 상호명, dtype: int64
본점     2134
2호점    1450
1호점     585
강남점     537
일산점     488
중앙점     442
평택점     432
부평점     409
구미점     404
안산점     392
Name: 지점명, dtype: int64
Q    914826
D    890294
F    431254
R    208809
S     91185
L     84655
N     75941
O     42835
P      1397
Name: 상권업종대분류코드, dtype: int64
음식          914826
소매          890294
생활서비스       431254
학문/교육       208809
의료           91185
부동산          84655
관광/여가/오락     75941
숙박           42835
스포츠           1397
Name: 상권업종대분류명, dtype: int64
Q01    336385
F01    198957
D03    159976
D05    145403
Q09    122921
D01    

서울특별시 강남구 역삼동      12102
경기도 안양시 만안구 안양동    11700
경기도 시흥시 정왕동        11365
인천광역시 부평구 부평동      10667
인천광역시 남구 주안동        9858
경기도 안산시 단원구 고잔동     9795
인천광역시 남동구 구월동       9636
서울특별시 강남구 대치동       8540
경기도 부천시 중동          8442
경기도 수원시 팔달구 인계동     8251
Name: 법정동명전체, dtype: int64
서울특별시 강남구       47080
경기도 부천시         38710
제주특별자치도 제주시     33548
경기도 화성시         31369
대전광역시 서구        31075
경기도 성남시 분당구     30409
인천광역시 남동구       29637
전라북도 전주시 완산구    29182
울산광역시 남구        28989
경상북도 구미시        28781
Name: 시군구명전체, dtype: int64


In [51]:
df['법정동코드'] == df['지번코드']

0         False
1         False
2         False
3         False
4         False
          ...  
769293    False
769294    False
769295    False
769296    False
769297    False
Length: 2741196, dtype: bool

In [52]:
df[df['지번코드'] == 4812312700200450001].head(5)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,...,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,경도_2,경도_1,경도_0,위도_2,위도_1,위도_0,법정동명전체,시군구명전체
531,21196813,림스치과의원,NaN,S,의료,S01,병원,S01B04,치과의원,Q86202,치과 의원,48,경상남도,48123,창원시 성산구,4812353000,상남동,4812312700,상남동,...,nan,4812312700100450001030595,성원아파트,경상남도 창원시 성산구 원이대로 774,642778,51499,NaN,5,NaN,129,35,129,129,129,35,35,35,경상남도 창원시 성산구 상남동,경상남도 창원시 성산구
1432,23296423,커피빈,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,48,경상남도,48123,창원시 성산구,4812353000,상남동,4812312700,상남동,...,nan,4812312700100450001032421,성원아파트,경상남도 창원시 성산구 원이대로 774,642778,51499,NaN,2,NaN,129,35,129,129,129,35,35,35,경상남도 창원시 성산구 상남동,경상남도 창원시 성산구
1449,23230559,루디아커텐,NaN,D,소매,D07,가정/주방/인테리어,D07A01,인테리어/욕실용품/커튼,G47599,그외 기타 분류안된 가정용품 소매업,48,경상남도,48123,창원시 성산구,4812353000,상남동,4812312700,상남동,...,nan,4812312700100450001032421,성원아파트,경상남도 창원시 성산구 원이대로 774,642778,51499,NaN,2,NaN,129,35,129,129,129,35,35,35,경상남도 창원시 성산구 상남동,경상남도 창원시 성산구
1628,24474501,알파플러스,스카이점,D,소매,D08,사무/문구/컴퓨터,D08A01,사무/문구용품,G47612,문구용품 소매업,48,경상남도,48123,창원시 성산구,4812353000,상남동,4812312700,상남동,...,nan,4812312700100450001030595,성원아파트,경상남도 창원시 성산구 원이대로 774,642778,51499,NaN,1,NaN,129,35,129,129,129,35,35,35,경상남도 창원시 성산구 상남동,경상남도 창원시 성산구
1666,25008964,블루,NaN,D,소매,D01,음/식료품소매,D01A01,식료품점,G47219,기타 식료품 소매업,48,경상남도,48123,창원시 성산구,4812353000,상남동,4812312700,상남동,...,nan,4812312700100450001030595,성원아파트,경상남도 창원시 성산구 원이대로 774,642778,51499,NaN,3,NaN,129,35,129,129,129,35,35,35,경상남도 창원시 성산구 상남동,경상남도 창원시 성산구


In [53]:
import pandas as pd
import numpy as np
import os
import logging
import numpy as np
from gensim.models.poincare import PoincareModel
from gensim.test.utils import datapath
import pickle
import sys

In [303]:
for col in ['지번코드', '법정동코드', '시군구코드', '시도코드']:
    df[col] = df[col].astype('Int64')

In [304]:
d_bdongs = dict(zip(df['법정동코드'], df['시도명'] + ' '+ df['시군구명'] + ' ' + df['법정동명']))
d_sis = dict(zip(df['시군구코드'], df['시도명'] + ' '+ df['시군구명']))
d_dos = dict(zip(df['시도코드'], df['시도명']))

In [305]:
d_bdong_si = dict(zip(df['법정동코드'].apply(lambda x: d_bdongs[x]), df['시군구코드'].apply(lambda x: d_sis[x])))
d_si_do = dict(zip(df['시군구코드'].apply(lambda x: d_sis[x]), df['시도코드'].apply(lambda x: d_dos[x])))
d_do_root = {do:0 for do in set(d_si_do.values())}

In [57]:
# d_add_bdong = dict(zip(df['지번코드'], df['법정동코드']))
# d_bdong_si = dict(zip(df['법정동코드'], df['시군구코드']))
# d_si_do = dict(zip(df['시군구코드'], df['시도코드']))
# d_do_root = {do:0 for do in set(d_si_do.values())}

In [306]:
hierarchical_list = []
for d in [d_do_root, d_si_do, d_bdong_si]: # d_add_bdong, d_bdong_si, 
    hierarchical_list.extend(d.items())
    
len(hierarchical_list)

5274

In [59]:
hierarchical_list

[('제주특별자치도', 0),
 ('충청남도', 0),
 ('인천광역시', 0),
 ('광주광역시', 0),
 ('서울특별시', 0),
 ('충청북도', 0),
 ('대전광역시', 0),
 ('경기도', 0),
 ('울산광역시', 0),
 ('강원도', 0),
 ('경상북도', 0),
 ('세종특별자치시', 0),
 ('대구광역시', 0),
 ('부산광역시', 0),
 ('경상남도', 0),
 ('전라북도', 0),
 ('전라남도', 0),
 ('서울특별시 강남구', '서울특별시'),
 ('서울특별시 금천구', '서울특별시'),
 ('부산광역시 사상구', '부산광역시'),
 ('서울특별시 송파구', '서울특별시'),
 ('서울특별시 광진구', '서울특별시'),
 ('서울특별시 강서구', '서울특별시'),
 ('서울특별시 중구', '서울특별시'),
 ('서울특별시 서초구', '서울특별시'),
 ('서울특별시 성북구', '서울특별시'),
 ('서울특별시 구로구', '서울특별시'),
 ('서울특별시 영등포구', '서울특별시'),
 ('서울특별시 서대문구', '서울특별시'),
 ('서울특별시 동대문구', '서울특별시'),
 ('서울특별시 노원구', '서울특별시'),
 ('부산광역시 동래구', '부산광역시'),
 ('부산광역시 부산진구', '부산광역시'),
 ('서울특별시 동작구', '서울특별시'),
 ('부산광역시 수영구', '부산광역시'),
 ('부산광역시 동구', '부산광역시'),
 ('서울특별시 성동구', '서울특별시'),
 ('부산광역시 중구', '부산광역시'),
 ('부산광역시 영도구', '부산광역시'),
 ('서울특별시 관악구', '서울특별시'),
 ('서울특별시 강동구', '서울특별시'),
 ('부산광역시 강서구', '부산광역시'),
 ('서울특별시 양천구', '서울특별시'),
 ('서울특별시 종로구', '서울특별시'),
 ('부산광역시 해운대구', '부산광역시'),
 ('부산광역시 금정구', '부산광역시'),
 ('부산광역시 연제구', '부산광역시'),

In [60]:
def embedding(relations,size,epochs):
    model = PoincareModel(relations,size=size,negative=5)
    model.train(epochs=epochs,batch_size=64,print_every=1)
    return model

In [61]:
size = 100
epochs = 10
m = embedding(hierarchical_list,size=int(size),epochs=int(epochs))

In [62]:
m.kv.ancestors('대전광역시 유성구 도룡동')

['대전광역시 유성구 둔곡동',
 '대전광역시 유성구 원내동',
 '대전광역시 유성구 어은동',
 '대전광역시 유성구 봉산동',
 '대전광역시',
 0]

In [63]:
m.kv.most_similar('대전광역시 유성구',topn=10)

[('대전광역시 유성구 봉산동', 0.061115537201714365),
 ('대전광역시 유성구 어은동', 0.06395893007361192),
 ('대전광역시 유성구 성북동', 0.06864930151740443),
 ('대전광역시 유성구 원내동', 0.07328456601168429),
 ('대전광역시 유성구 하기동', 0.07865784609268178),
 ('대전광역시 유성구 도룡동', 0.07978495289534365),
 ('대전광역시 유성구 대동', 0.08107285640510792),
 ('대전광역시 유성구 자운동', 0.0811852624873982),
 ('대전광역시 유성구 안산동', 0.08388009618679676),
 ('대전광역시 유성구 죽동', 0.08544465396112924)]

In [64]:
m.kv.get_vector('대전광역시')

array([ 9.08120812e-03,  1.72159855e-02,  9.64200048e-03,  3.80633504e-02,
       -1.63920975e-02,  3.07162888e-02, -6.61217193e-04,  2.21277056e-02,
        5.86816899e-03, -3.58256477e-02, -4.92592990e-03, -2.92858822e-02,
        8.67207045e-03, -2.58991272e-03,  1.60694297e-02,  1.01934414e-02,
        2.82631801e-02, -2.99078538e-03, -1.69673325e-02, -5.94331729e-05,
        2.56530905e-02, -5.45151813e-03,  5.47394440e-04,  9.35993967e-03,
        2.03652185e-02,  5.79199304e-03,  3.10187198e-02, -1.56460080e-02,
       -2.60988764e-02,  6.81517754e-03,  1.40631069e-03, -1.24517122e-05,
        1.51516180e-02, -6.23752430e-03, -1.92130045e-02,  2.59298065e-03,
       -1.88246022e-02,  1.33503306e-02, -5.18396165e-03, -3.15191168e-04,
       -3.43837411e-03,  8.96692905e-03, -7.89201108e-03, -3.34732882e-02,
        9.43189390e-03, -6.25672281e-03, -6.24250078e-03,  5.42247254e-03,
       -1.64864166e-02,  2.17718854e-02, -1.48218113e-02,  1.34445400e-02,
       -5.68839213e-03,  

In [65]:
max(df.경도)

131.86562411791098

In [307]:
# latitute and longitude

df.loc[:, '경도_2'] = df['경도'].apply(lambda x: round(x, 2))
df.loc[:, '경도_1'] = df['경도'].apply(lambda x: round(x, 1))
df.loc[:, '경도_0'] = df['경도'].apply(lambda x: round(x, 0))
df.loc[:, '위도_2'] = df['위도'].apply(lambda x: round(x, 2))
df.loc[:, '위도_1'] = df['위도'].apply(lambda x: round(x, 1))
df.loc[:, '위도_0'] = df['위도'].apply(lambda x: round(x, 0))

In [308]:
latlng = set()
largest_latlng = set()

for i in range(0, 2):
    lat1 = df['경도_'+str(i+1)]
    lng1 = df['위도_'+str(i+1)]
    lat2 = df['경도_'+str(i)]
    lng2 = df['위도_'+str(i)]

    for at1, ng1, at2, ng2 in zip(lat1, lng1, lat2, lng2):
        tup1 = tuple((ng1, at1))
        tup2 = tuple((ng2, at2))
        latlng.add((tup1, tup2))

        if i == 0:
            largest_latlng.add(tup2)

In [309]:
df['법정동명전체'] = (df['시도명'] + ' '+ df['시군구명'] + ' ' + df['법정동명'])
df['시군구명전체'] = (df['시도명'] + ' '+ df['시군구명'])

latlng_list = list(latlng) + (list(set((x, (y,z)) for x, y, z in df[['법정동명전체','위도_1', '경도_1']].dropna(axis=0).values)) 
                              + list(set((x, (y,z)) for x, y, z in df[['법정동명전체','위도_2', '경도_2']].dropna(axis=0).values))
                              + list(set((x, (y,z)) for x, y, z in df[['시군구명전체','위도_1', '경도_1']].dropna(axis=0).values))
                              + list(set((x, (y,z)) for x, y, z in df[['시군구명전체','위도_2', '경도_2']].dropna(axis=0).values))
                              + list(set((x, (y,z)) for x, y, z in df[['시도명','위도_1', '경도_1']].dropna(axis=0).values)))

for item in largest_latlng:
    latlng_list.append((0, item))

print(len(latlng_list)) #16198개

hierarchical_list += latlng_list
print(len(hierarchical_list))     

153994
159268


In [69]:
hierarchical_list[-500:]

[('경상북도', (36.1, 129.4)),
 ('경상북도', (36.6, 129.1)),
 ('경상북도', (36.1, 129.3)),
 ('경상북도', (36.1, 129.0)),
 ('경상북도', (36.6, 129.4)),
 ('경상북도', (36.1, 129.2)),
 ('경상북도', (36.2, 129.1)),
 ('경상북도', (36.4, 129.3)),
 ('경상북도', (36.2, 129.3)),
 ('경상북도', (36.7, 129.3)),
 ('경기도', (37.2, 126.6)),
 ('경기도', (37.7, 126.6)),
 ('강원도', (37.3, 129.1)),
 ('경상북도', (35.9, 129.4)),
 ('전라북도', (35.7, 126.8)),
 ('전라북도', (35.5, 126.5)),
 ('경상북도', (36.5, 129.1)),
 ('경상북도', (36.0, 129.1)),
 ('경상북도', (36.0, 129.6)),
 ('경상북도', (36.4, 128.3)),
 ('경상북도', (36.9, 128.8)),
 ('경상북도', (36.4, 128.8)),
 ('경상북도', (36.4, 128.7)),
 ('경상북도', (36.4, 128.2)),
 ('경상북도', (36.9, 128.7)),
 ('제주특별자치도', (33.2, 126.2)),
 ('제주특별자치도', (34.0, 126.3)),
 ('경기도', (37.9, 127.6)),
 ('경기도', (37.9, 127.1)),
 ('경기도', (37.4, 127.1)),
 ('경기도', (37.4, 127.6)),
 ('경상남도', (35.0, 128.5)),
 ('경상남도', (35.5, 128.5)),
 ('경상남도', (35.5, 128.0)),
 ('경상남도', (35.0, 128.0)),
 ('충청북도', (36.8, 127.7)),
 ('충청북도', (36.2, 127.8)),
 ('충청북도', (36.7, 127.3)),
 ('충청북도', (36

In [70]:
hierarchical_list[-5000:]

[('충청북도 진천군', (36.76, 127.42)),
 ('충청북도 충주시', (36.99, 127.74)),
 ('충청북도 충주시', (36.99, 127.99)),
 ('충청북도 충주시', (36.93, 127.92)),
 ('경기도 여주시', (37.32, 127.54)),
 ('경기도 여주시', (37.18, 127.68)),
 ('충청남도 논산시', (36.14, 127.3)),
 ('충청남도 논산시', (36.28, 127.1)),
 ('제주특별자치도 제주시', (33.44, 126.48)),
 ('제주특별자치도 제주시', (33.5, 126.67)),
 ('제주특별자치도 제주시', (33.42, 126.38)),
 ('제주특별자치도 제주시', (33.44, 126.73)),
 ('경상남도 창원시 의창구', (35.36, 128.65)),
 ('제주특별자치도 제주시', (33.33, 126.16)),
 ('전라남도 나주시', (34.92, 126.74)),
 ('울산광역시 울주군', (35.52, 129.23)),
 ('경상북도 봉화군', (36.96, 128.94)),
 ('전라남도 신안군', (34.68, 125.41)),
 ('충청북도 보은군', (36.39, 127.75)),
 ('강원도 평창군', (37.66, 128.69)),
 ('강원도 평창군', (37.47, 128.26)),
 ('충청남도 아산시', (36.86, 126.93)),
 ('경상남도 합천군', (35.58, 128.25)),
 ('강원도 태백시', (37.13, 128.98)),
 ('강원도 태백시', (37.23, 128.94)),
 ('강원도 태백시', (37.13, 128.97)),
 ('강원도 태백시', (37.13, 128.96)),
 ('강원도 태백시', (37.26, 128.96)),
 ('강원도 태백시', (37.26, 128.98)),
 ('경상북도 예천군', (36.75, 128.49)),
 ('경상남도 창녕군', (35.53, 128.49)),
 

In [275]:
'경기도 성남시' in [e[0] for e in hierarchical_list if isinstance(e[0], str) and e[0][-1] == '구']

False

In [71]:
size = 100
epochs = 30
m2 = embedding(hierarchical_list,size=int(size),epochs=int(epochs))

In [76]:
def most_similar(model, query, topn):
    result = []
    cand = model.kv.most_similar(query,topn*10)
    for can in cand:
        if isinstance(can[0], str):
            result.append(can)
    return result[:topn]

In [86]:
def most_similar2(model, query, topn):
    result = []
    cand = model.kv.most_similar(query,topn*10)
    for can in cand:
        if isinstance(can[0], str):
            if can[0][-1] == query[-1]:
                result.append(can)
    return result[:topn]

In [148]:
[e[0] for e in hierarchical_list if isinstance(e[0], str) and '강원도 평창군' in e[0]]

['강원도 평창군',
 '강원도 평창군 진부면',
 '강원도 평창군 평창읍',
 '강원도 평창군 미탄면',
 '강원도 평창군 용평면',
 '강원도 평창군 대화면',
 '강원도 평창군 봉평면',
 '강원도 평창군 대관령면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 평창읍',
 '강원도 평창군 평창읍',
 '강원도 평창군 용평면',
 '강원도 평창군 용평면',
 '강원도 평창군 대관령면',
 '강원도 평창군 대관령면',
 '강원도 평창군 대관령면',
 '강원도 평창군 대관령면',
 '강원도 평창군 봉평면',
 '강원도 평창군 봉평면',
 '강원도 평창군 봉평면',
 '강원도 평창군 진부면',
 '강원도 평창군 진부면',
 '강원도 평창군 미탄면',
 '강원도 평창군 대화면',
 '강원도 평창군 대화면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 평창읍',
 '강원도 평창군 평창읍',
 '강원도 평창군 용평면',
 '강원도 평창군 용평면',
 '강원도 평창군 대관령면',
 '강원도 평창군 대관령면',
 '강원도 평창군 봉평면',
 '강원도 평창군 봉평면',
 '강원도 평창군 진부면',
 '강원도 평창군 진부면',
 '강원도 평창군 진부면',
 '강원도 평창군 진부면',
 '강원도 평창군 진부면',
 '강원도 평창군 미탄면',
 '강원도 평창군 미탄면',
 '강원도 평창군 대화면',
 '강원도 평창군 대화면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 방림면',
 '강원도 평창군 대관령면',
 '강원도 평창군 평창읍',
 '강원도 평창군 평창읍',
 '강원도 평창군 용평면',
 '강원도 평창군 용평면',
 '강원

In [160]:
most_similar(m2,'경기도 성남시 분당구 정자동',10000)

[('경기도 성남시 분당구 수내동', 0.4289041634540217),
 ('경기도 성남시 분당구 궁내동', 0.5057546726630546),
 ('경기도 성남시 분당구 백현동', 0.5634441887065664),
 ('경기도 성남시 분당구 이매동', 0.5915119451219025),
 ('경기도 성남시 분당구 판교동', 0.6321662880658588),
 ('경기도 성남시 분당구 서현동', 0.6398304647800531),
 ('경기도 성남시 분당구 하산운동', 0.6859605956220978),
 ('경기도 성남시 분당구', 0.6935229758721055),
 ('경기도 성남시 분당구 운중동', 0.7455345719832759),
 ('경기도 성남시 분당구 율동', 0.9123031878685148),
 ('경기도 성남시 분당구 삼평동', 0.9250339985037606),
 ('경기도 성남시 분당구 금곡동', 1.0159431131835277),
 ('경기도 성남시 분당구 대장동', 1.1383871707338613),
 ('경기도 성남시 분당구 분당동', 1.8379629733792253),
 ('경기도 성남시 분당구 야탑동', 1.9151510724338217),
 ('경기도 성남시 분당구 석운동', 2.118229368107524),
 ('경기도 성남시 수정구 사송동', 2.4425098621314265),
 ('경기도 성남시 중원구 여수동', 2.458788881004722),
 ('경기도 성남시 분당구 동원동', 2.655368303363486),
 ('경기도 성남시 수정구 금토동', 2.7101808803351646),
 ('경기도 성남시 수정구 상적동', 2.848378503185644),
 ('경기도 성남시 수정구 시흥동', 2.849847576759607),
 ('경기도 성남시 수정구 고등동', 2.9456161405351597),
 ('경기도 성남시 중원구 성남동', 3.006614269044302),
 (

In [74]:
m2.kv.get_vector('대전광역시')

array([-0.0991057 , -0.12776225, -0.01608572,  0.11535728, -0.01205722,
       -0.00824891, -0.0019516 , -0.04374997, -0.01826685,  0.006629  ,
        0.02039487, -0.11407817,  0.00659377, -0.12057734, -0.14965614,
        0.03227483,  0.09440104, -0.19765511, -0.00623371,  0.097401  ,
        0.11464548, -0.02005068,  0.00095557,  0.05110917,  0.05246248,
       -0.02925154,  0.08416108,  0.0312601 ,  0.03511026,  0.08068204,
       -0.01119383, -0.02127617,  0.11179139, -0.05987794, -0.09690967,
       -0.08767697, -0.15200517,  0.05242372,  0.0621465 ,  0.07512932,
       -0.12919707,  0.05744385,  0.03072036, -0.00698287, -0.0733485 ,
        0.05203092,  0.11643963, -0.10670456,  0.01135298,  0.14599862,
       -0.03036175,  0.09432388,  0.15351757, -0.02611173,  0.05549385,
       -0.03786422, -0.07041446,  0.03300167,  0.13623393,  0.06425779,
        0.05780508, -0.05462389, -0.07572857, -0.04168082, -0.06366149,
        0.00034509,  0.17174099,  0.05572511, -0.13074153,  0.09

In [164]:
EMB_SOURCE = 'district+latlng'

import pickle
pickle.dump(m2, open('poincare_embedding_result_'+str(size)+'+'+str(epochs)+EMB_SOURCE+'.p', 'wb'))

### 행정동 추가

In [311]:
df['행정동명전체'] = (df['시도명'] + ' '+ df['시군구명'] + ' ' + df['행정동명'])

In [312]:
df['행정동명전체'].nunique(), df['행정동코드'].nunique()

(3582, 3587)

In [313]:
df['행정동명전체'] = (df['시도명'] + ' '+ df['시군구명'] + ' ' + df['행정동명'] + ' (행정동)')


dong_pairs = list((x,y) for x, y in df[['행정동명전체', '법정동명전체']].dropna(axis=0).drop_duplicates().values)
hierarchical_list += dong_pairs

latlng_list2 = (list(set((x, (y,z)) for x, y, z in df[['행정동명전체','위도_1', '경도_1']].dropna(axis=0).values)) 
                + list(set((x, (y,z)) for x, y, z in df[['행정동명전체','위도_2', '경도_2']].dropna(axis=0).values))
               )

print(len(dong_pairs))
print(len(latlng_list2)) 

hierarchical_list += latlng_list2
print(len(hierarchical_list))      

6153
57510
222931


In [314]:
size = 100
epochs = 30
m3 = embedding(hierarchical_list,size=int(size),epochs=int(epochs))

In [316]:
EMB_SOURCE = 'district(행정+법정)+latlng'

import pickle
pickle.dump(m3, open('poincare_embedding_result_'+str(size)+'_'+str(epochs)+'+'+EMB_SOURCE+'.p', 'wb'))

### 도로명 추가

In [355]:
df[df['도로명'] == '인천광역시 옹진군 영흥면 영흥로'][['행정동명', '법정동명', '도로명', '도로명주소']]

,행정동명,법정동명,도로명,도로명주소
29,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 109-10
82,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 109-10
2563,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 109-10
2628,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 109-10
2699,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 109-10
...,...,...,...,...
495609,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 256
496854,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 144
505883,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 299
515725,영흥면,영흥면,인천광역시 옹진군 영흥면 영흥로,인천광역시 옹진군 영흥면 영흥로 679-1


In [351]:
df['도로명'].sample(n=20)

597884                 충청남도 아산시 시장남길
52073     경상북도 포항시 북구 죽장면 새마을로3599번길
328726        인천광역시 옹진군 영흥면 영흥로176번길
213154                 충청남도 아산시 용화고길
66557                 서울특별시 은평구 진관2로
32331               전라남도 구례군 구례읍 중앙로
301579                 광주광역시 남구 서문대로
320537                서울특별시 마포구 와우산로
406720             충청남도 예산군 예산읍 신례원로
558938         경상남도 창원시 의창구 우곡로217번길
515127              경기도 고양시 일산동구 성석로
737911          경기도 김포시 양촌읍 양곡2로30번길
285865            경상북도 안동시 서후면 성곡제일길
397461           경기도 수원시 장안구 만석로19번길
3631           경기도 고양시 일산동구 호수로430번길
61421           경상북도 포항시 남구 상공로134번길
360946                부산광역시 사하구 낙동대로
256983                부산광역시 사하구 낙동대로
711420               경기도 안양시 만안구 양화로
111895              충청남도 천안시 서북구 오성로
Name: 도로명, dtype: object

In [358]:
df['도로명'].nunique()

110722

In [360]:
df['행정동명전체'].nunique()

3582

In [362]:
df[['법정동명전체', '도로명']].dropna(axis=0).drop_duplicates().shape

(123826, 2)

In [368]:
df[['도로명', '위도_2', '경도_2']].dropna(axis=0).drop_duplicates().shape

(170915, 3)

In [370]:
district_road_pairs = list((x,y) for x, y in df[['법정동명전체', '도로명']].dropna(axis=0).drop_duplicates().values)
hierarchical_list += district_road_pairs
latlng_list3 = list(set((x, (y,z)) for x, y, z in df[['도로명', '위도_2', '경도_2']].dropna(axis=0).values))

print(len(district_road_pairs))
print(len(latlng_list3)) 

hierarchical_list += latlng_list3
print(len(hierarchical_list))      

123826
170915
517672


In [371]:
size = 100
epochs = 30
m4 = embedding(hierarchical_list,size=int(size),epochs=int(epochs))

In [372]:
EMB_SOURCE = 'district(행정+법정)+도로명+latlng'

import pickle
pickle.dump(m4, open('poincare_embedding_result_'+str(size)+'_'+str(epochs)+'+'+EMB_SOURCE+'.p', 'wb'))

In [416]:
### 지번, 도로명주소, 업종 추가

In [419]:
df[['도로명주소', '지번주소']]

,도로명주소,지번주소
0,서울특별시 강남구 남부순환로 2933,서울특별시 강남구 대치동 604
1,서울특별시 강남구 선릉로93길 6,서울특별시 강남구 역삼동 696-42
2,서울특별시 금천구 가산로 34-6,서울특별시 금천구 독산동 162-1
3,부산광역시 사상구 괘감로 37,부산광역시 사상구 괘법동 578
4,서울특별시 송파구 가락로 71,서울특별시 송파구 석촌동 256
...,...,...
769293,제주특별자치도 제주시 일주서로 7810,제주특별자치도 제주시 연동 2414-2
769294,제주특별자치도 제주시 노연로 103,제주특별자치도 제주시 연동 273-43
769295,전라북도 군산시 대학로 77,전라북도 군산시 명산동 19-14
769296,경상남도 진주시 모덕로 223,경상남도 진주시 하대동 343-40


In [425]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도', '경도_2', '경도_1', '경도_0', '위도_2', '위도_1', '위도_0',
       '법정동명전체', '시군구명전체', '행정동명전체'],
      dtype='object')

In [431]:
df['상호지점명'] = df['상호명'] + ' ' + df['지점명']

In [434]:
set(df['상호명']).intersection(set(df['상호지점명']))

{nan, '롯데프리미엄아울렛 파주점', '메트로시티 현대백화점부산점', '보떼 평화점', '아트박스 일산점', '페리카나 용전점'}

In [443]:
df[df['상호명'].isna()].shape

(2, 49)

In [444]:
df[df['지점명'].isna()].shape

(2407872, 49)

In [441]:
df[df['상호지점명'].isna()].shape

(2407872, 49)

In [445]:
df.shape

(2741196, 49)

In [448]:
df[~df['지점명'].isna()][['상호명', '상호지점명']].dropna(axis=0).drop_duplicates().shape

(319757, 2)

* 지점명이 있으면(34만 건): 상호명 -> 상호지점명 -> (지번주소, 도로명주소)  
* 지점명이 없으면(240만 건): 상호명 -> (지번주소, 도로명주소)

In [480]:
f1 = list(((x,y),(y,z),(y,w)) for x,y,z,w in df[~df['지점명'].isna()][['상호명', '상호지점명', '도로명주소', '지번주소']].dropna(axis=0).drop_duplicates().values)
f1 = [item for sublist in f1 for item in sublist]

f2 = list(((x,y),(y,z)) for x,y,z in df[df['지점명'].isna()][['상호명', '도로명주소', '지번주소']].dropna(axis=0).drop_duplicates().values)
f2 = [item for sublist in f2 for item in sublist]

f3 = list((x,y) for x, y in df[['도로명', '도로명주소']].dropna(axis=0).drop_duplicates().values)
f4 = list((x,y) for x, y in df[['법정동명', '지번주소']].dropna(axis=0).drop_duplicates().values)

f5 = list(((x,y),(y,z),(z,w)) for x, y, z, w in df[['상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '상호명']].dropna(axis=0).drop_duplicates().values)
f5 = [item for sublist in f5 for item in sublist]

len(f1), len(f2), len(f3), len(f4), len(f5)

(999153, 4805912, 991559, 987608, 5178540)

In [481]:
f6  = list((((x,y),z), ((x,y),w)) for x, y, z, w in df[['위도_2', '경도_2', '도로명주소', '지번주소']].dropna(axis=0).drop_duplicates().values)
f6 = [item for sublist in f6 for item in sublist]
len(f6)

2016730

In [482]:
f7 = list((0, item)for item in set(df['상권업종대분류명']))

In [ ]:

print(len(district_road_pairs))
print(len(latlng_list3)) 

hierarchical_list += latlng_list3
print(len(hierarchical_list))      

In [ ]:
size = 100
epochs = 30
m4 = embedding(hierarchical_list,size=int(size),epochs=int(epochs))

In [ ]:
EMB_SOURCE = 'district(행정+법정)+도로명+latlng'

import pickle
pickle.dump(m4, open('poincare_embedding_result_'+str(size)+'_'+str(epochs)+'+'+EMB_SOURCE+'.p', 'wb'))

### 출력

In [486]:
pd.set_option('display.float_format', '{:.3f}'.format)

In [487]:
with open("poincare_embedding_result_100+30district+latlng.p", "rb") as f:
    m2 = pickle.load(f)

In [488]:
def make_csv(df, model, level):
    index = []
    data = []
    for i in df[level].dropna(axis=0).unique():
        data.append(model.kv.get_vector(i))
        index.append(i)
    pd.DataFrame(data=data,index =index).to_csv('embedding+'+level+'.csv',index=True)

In [489]:
make_csv(df, m2, '시군구명전체')
make_csv(df, m2, '법정동명전체')

In [490]:
check1 = pd.read_csv("embedding+법정동명전체.csv",index_col=0)

In [491]:
cembs = {}
for i in check1.index:
    cembs[i] = np.array(check1.loc[i])

In [492]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10):
    distance = {}
    for i in cembs:
        distance[i] = cosine(cembs[query], cembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [493]:
print_top_n_similar('서울특별시 강남구 역삼동', topn=10)

[('서울특별시 강남구 역삼동', 0.0),
 ('서울특별시 강남구 도곡동', 0.00034705008069657595),
 ('서울특별시 서초구 염곡동', 0.0034597326776079074),
 ('서울특별시 서초구 신원동', 0.004609482507057505),
 ('서울특별시 강남구 신사동', 0.004808649497375028),
 ('서울특별시 강남구 청담동', 0.005062099674231657),
 ('서울특별시 강남구 압구정동', 0.005399437976143173),
 ('서울특별시 강남구 논현동', 0.0058390536029466755),
 ('서울특별시 서초구 원지동', 0.006779615730269195),
 ('서울특별시 서초구 양재동', 0.007136003041105621)]

In [495]:
print_top_n_similar('대전광역시 유성구 구성동', topn=10)

[('대전광역시 유성구 구성동', 0.0),
 ('대전광역시 유성구 관평동', 2.4905380260453747e-05),
 ('대전광역시 유성구 탑립동', 2.96965698958207e-05),
 ('대전광역시 유성구 송강동', 3.139385792316762e-05),
 ('대전광역시 유성구 도룡동', 4.5154039977068905e-05),
 ('대전광역시 유성구 원촌동', 4.795784686184579e-05),
 ('대전광역시 유성구 방현동', 5.7724033854511525e-05),
 ('대전광역시 유성구 덕진동', 6.025953724975164e-05),
 ('대전광역시 유성구 화암동', 6.733634271294253e-05),
 ('대전광역시 유성구 용산동', 8.483810468229436e-05)]

In [496]:
print_top_n_similar('경기도 성남시 분당구 정자동')

[('경기도 성남시 분당구 정자동', 0.0),
 ('경기도 성남시 분당구 궁내동', 5.407969881354724e-05),
 ('경기도 성남시 분당구 백현동', 8.287912284543619e-05),
 ('경기도 성남시 분당구 수내동', 9.850540402678298e-05),
 ('경기도 성남시 분당구 하산운동', 0.0001152969169296858),
 ('경기도 성남시 분당구 서현동', 0.00011546424496633456),
 ('경기도 성남시 분당구 이매동', 0.00015036160269699117),
 ('경기도 성남시 분당구 판교동', 0.0003164222694661545),
 ('경기도 성남시 분당구 삼평동', 0.00041646277453022407),
 ('경기도 성남시 분당구 운중동', 0.0009836712059955177)]

In [499]:
with open("poincare_embedding_result_100_30+district(행정+법정)+latlng.p", "rb") as f:
    m3 = pickle.load(f)

In [500]:
m3.kv.most_similar('대전광역시 유성구 노은3동 (행정동)',topn=100)

[('대전광역시 유성구 지족동', 0.11696619647197988),
 ('대전광역시 유성구 노은1동 (행정동)', 0.48615748957368143),
 ('대전광역시 유성구 노은동', 0.506267430690338),
 ('대전광역시 유성구 반석동', 0.5327012699526824),
 ('대전광역시 유성구 안산동', 0.6165695607920282),
 ('대전광역시 유성구 죽동', 0.6218657899473111),
 ('대전광역시 유성구 갑동', 0.6351965819888812),
 ('대전광역시 유성구 외삼동', 0.6382892978640478),
 ('대전광역시 유성구 하기동', 0.6988902781773074),
 ('대전광역시 유성구 노은2동 (행정동)', 0.7161885550914946),
 ('대전광역시 유성구 수남동', 0.787643051616981),
 ('대전광역시 유성구 장대동', 1.0693500170771855),
 ('대전광역시 유성구 신봉동', 1.1179058484927182),
 ('대전광역시 유성구 구암동', 1.1618649001019214),
 ('대전광역시 유성구 궁동', 1.1926016422349373),
 ('대전광역시 유성구 덕명동', 1.312199664535944),
 ('대전광역시 유성구 자운동', 1.3167139569441184),
 ('대전광역시 유성구 방현동', 1.3462738050516547),
 ('대전광역시 유성구', 1.34715172690798),
 ('대전광역시 유성구 복용동', 1.3734315870657476),
 ('대전광역시 유성구 추목동', 1.3897155013073477),
 ('대전광역시 유성구 상대동', 1.4265045760010557),
 ('대전광역시 유성구 구성동', 1.433040415902798),
 ('대전광역시 유성구 신성동', 1.4803387049061563),
 ('대전광역시 유성구 어은동', 1.4904130768262676

In [501]:
make_csv(df, m3, '시군구명전체')
make_csv(df, m3, '법정동명전체')
make_csv(df, m3, '행정동명전체')

In [502]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10, emb_rslt=''):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [503]:
emb_rslt = pd.read_csv("embedding+행정동명전체.csv",index_col=0)

In [504]:
print_top_n_similar('대전광역시 유성구 신성동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 신성동 (행정동)', 0.0),
 ('대전광역시 유성구 온천2동 (행정동)', 0.002657204746375541),
 ('대전광역시 유성구 관평동 (행정동)', 0.006685385360739704),
 ('대전광역시 유성구 노은1동 (행정동)', 0.006717428686011773),
 ('대전광역시 유성구 노은2동 (행정동)', 0.006744247460650832),
 ('대전광역시 유성구 노은3동 (행정동)', 0.007068288152155766),
 ('대전광역시 유성구 구즉동 (행정동)', 0.008834551670480417),
 ('대전광역시 유성구 온천1동 (행정동)', 0.009772985374151277),
 ('대전광역시 유성구 전민동 (행정동)', 0.011864976326499077),
 ('대전광역시 유성구 원신흥동 (행정동)', 0.015278619539778937)]

In [505]:
print_top_n_similar('경기도 성남시 수정구 신흥1동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('경기도 성남시 수정구 신흥1동 (행정동)', 0.0),
 ('경기도 성남시 수정구 신흥3동 (행정동)', 0.0011699633817944255),
 ('경기도 성남시 수정구 태평3동 (행정동)', 0.001420437034571509),
 ('경기도 성남시 수정구 수진1동 (행정동)', 0.002278253209070269),
 ('경기도 성남시 수정구 태평4동 (행정동)', 0.0024055078904364757),
 ('경기도 성남시 수정구 수진2동 (행정동)', 0.0025100315755293146),
 ('경기도 성남시 수정구 태평2동 (행정동)', 0.0025237677386418245),
 ('경기도 성남시 수정구 태평1동 (행정동)', 0.0033429088726696365),
 ('경기도 성남시 수정구 신흥2동 (행정동)', 0.005967442548124313),
 ('경기도 성남시 수정구 고등동 (행정동)', 0.008226930711372593)]

In [506]:
with open("poincare_embedding_result_100_30+district(행정+법정)+도로명+latlng.p", "rb") as f:
    m4 = pickle.load(f)

In [507]:
m4.kv.most_similar('부산광역시 사하구 낙동대로',topn=10)

[('부산광역시 사하구 승학로', 0.4822714690316676),
 ('부산광역시 사하구 제석로18번길', 0.5135692814129723),
 ('부산광역시 사하구 동매로', 0.5303985675274213),
 ('부산광역시 사하구 괴정로', 0.5539521550251503),
 ('부산광역시 사하구 제석로', 0.6226081402654269),
 ('부산광역시 사하구 낙동대로410번길', 0.6813411595311761),
 ('부산광역시 사하구 낙동대로436번길', 0.6953391704796298),
 ('부산광역시 사하구 낙동대로407번길', 0.702109738235207),
 ('부산광역시 사하구 승학로17번길', 0.720289262024862),
 ('부산광역시 사하구 제석로59번안길', 0.7330452188621798)]

In [375]:
m4.kv.most_similar('전라남도 구례군 구례읍 중앙로',topn=10)

[('전라남도 구례군 구례읍 봉성산길', 0.019374836683656596),
 ('전라남도 구례군 구례읍 백련길', 0.021330464555723544),
 ('전라남도 구례군 구례읍 5일시장큰길', 0.02575553702651517),
 ('전라남도 구례군 구례읍 봉북3길', 0.02595841550561294),
 ('전라남도 구례군 구례읍 봉덕정길', 0.027153506902410588),
 ('전라남도 구례군 구례읍 5일시장길', 0.02851240330948692),
 ('전라남도 구례군 구례읍 봉동길', 0.030836844244861012),
 ('전라남도 구례군 구례읍 봉성산샛길', 0.0309114658488147),
 ('전라남도 구례군 구례읍 숲거리1길', 0.03170413875800573),
 ('전라남도 구례군 구례읍 5일시장작은길', 0.033425337307399355)]

In [410]:
def make_csv(df, model, level):
    index = []
    data = []
    for i in df[level].dropna(axis=0).unique():
        data.append(model.kv.get_vector(i).astype(np.float32))
        index.append(i)
    rslt = pd.DataFrame(data=data,index=index)
    rslt.to_csv('embedding+'+level.replace('전체','')+'.csv',index=True)

In [411]:
make_csv(df, m4, '시군구명전체')
make_csv(df, m4, '법정동명전체')
make_csv(df, m4, '행정동명전체')
make_csv(df, m4, '도로명')

In [378]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10, emb_rslt=''):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [379]:
emb_rslt = pd.read_csv("embedding+도로명.csv",index_col=0)

In [380]:
print_top_n_similar('부산광역시 사하구 낙동대로', topn=10, emb_rslt=emb_rslt)

[('부산광역시 사하구 낙동대로', 0.0),
 ('부산광역시 사하구 승학로', 0.015802638494118337),
 ('부산광역시 사하구 제석로18번길', 0.01655808705211581),
 ('부산광역시 사하구 동매로', 0.018345108305352964),
 ('부산광역시 사하구 괴정로', 0.02117502532938831),
 ('부산광역시 사하구 제석로', 0.024434767585734818),
 ('부산광역시 사하구 낙동대로436번길', 0.028940565138159124),
 ('부산광역시 사하구 낙동대로407번길', 0.03214763564723899),
 ('부산광역시 사하구 낙동대로410번길', 0.034144826602908185),
 ('부산광역시 사하구 승학로17번길', 0.038548895601949296)]

In [381]:
print_top_n_similar('대전광역시 유성구 엑스포로', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 엑스포로', 0.0),
 ('대전광역시 유성구 문지로', 0.0002718513287518487),
 ('대전광역시 유성구 전민로', 0.00038442923126769113),
 ('대전광역시 유성구 엑스포로540번길', 0.0007245345632788114),
 ('대전광역시 유성구 엑스포로466번길', 0.0008326375000698993),
 ('대전광역시 유성구 엑스포로538번길', 0.0008356208308727409),
 ('대전광역시 유성구 유성대로1689번길', 0.0008854344348454557),
 ('대전광역시 유성구 엑스포로446번길', 0.0008893499329751808),
 ('대전광역시 유성구 엑스포로295번길', 0.0009518559055993192),
 ('대전광역시 유성구 유성대로1628번길', 0.001476171812361371)]

In [589]:
with open("poincare_embedding_result_100_30+district(행정+법정)+도로명+지번도로명주소+업종+상호명+latlng.p", "rb") as f:
    m5 = pickle.load(f)

In [590]:
m5.kv.most_similar('부산광역시 사하구 낙동대로',topn=10)

[('부산광역시 사하구 괴정4동 (행정동)', 0.3925723574357385),
 ('부산광역시 사하구 승학로161번안길', 0.39434014894256514),
 ('부산광역시 사하구 당리동 (행정동)', 0.39934137902148004),
 ('부산광역시 사하구 괴정동', 0.4498593472962308),
 ('부산광역시 사하구 승학로', 0.462553074544191),
 ('부산광역시 사하구 낙동대로318번길', 0.48127301611728035),
 ('부산광역시 사하구 마하로48번길', 0.4904478142442992),
 ('부산광역시 사하구 오작로', 0.5130778242570186),
 ('부산광역시 사하구 당리동', 0.5164516475844598),
 ((35.1, 128.99), 0.5173901391488992)]

In [591]:
m5.kv.most_similar('페리카나 용전점',topn=10)

[('항아리칼국수용전', 0.5539394524930117),
 ('주섬주섬', 0.8971180162579928),
 ('주섬주섬 용전점', 0.9944195238963651),
 ('투마리치킨 한남대점', 1.0337060973945058),
 ('대전광역시 동구 계족로 481', 1.2520770865521953),
 ('대전광역시 동구 용전동 160-21', 1.2895715759690733),
 ('오케이', 1.8509256123425952),
 ('피부관리샵', 1.8614857893420056),
 ('황실노래주점', 1.8654932679200522),
 ('대전광역시 동구 동서대로1695번길', 1.8726206916111452)]

In [592]:
make_csv(df, m5, '시군구명전체')
make_csv(df, m5, '법정동명전체')
make_csv(df, m5, '행정동명전체')
make_csv(df, m5, '도로명')
make_csv(df, m5, '상호명')
make_csv(df, m5, '상호지점명')
make_csv(df, m5, '상권업종대분류명')
make_csv(df, m5, '상권업종중분류명')
make_csv(df, m5, '상권업종소분류명')
make_csv(df, m5, '도로명주소')
make_csv(df, m5, '지번주소')

In [601]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [597]:
emb_rslt = pd.read_csv("embedding+시군구명.csv",index_col=0)

In [602]:
print_top_n_similar('서울특별시 강남구', topn=10, emb_rslt=emb_rslt)

[('서울특별시 강남구', 0.0),
 ('서울특별시 송파구', 0.34727637437385483),
 ('서울특별시 서초구', 0.3968795838441066),
 ('경기도 성남시 수정구', 0.7272891407496148),
 ('서울특별시 용산구', 0.7415661470418881),
 ('경기도 연천군', 0.7456407562652763),
 ('울산광역시 남구', 0.7550280635048319),
 ('서울특별시 광진구', 0.7626552906740318),
 ('서울특별시 강동구', 0.7681690860050584),
 ('경기도 안성시', 0.7939815713191984)]

In [603]:
print_top_n_similar('대전광역시 유성구', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구', 0.0),
 ('충청남도 계룡시', 0.44828839816753996),
 ('대전광역시 대덕구', 0.4661577383580363),
 ('대전광역시 서구', 0.594167026715532),
 ('대구광역시 달서구', 0.7034313247469295),
 ('경상북도 영덕군', 0.7410603634659718),
 ('충청북도 청주시 서원구', 0.7943546374130585),
 ('경상남도 사천시', 0.7979637976083602),
 ('충청남도 공주시', 0.82080370545287),
 ('부산광역시 기장군', 0.8229349856952675)]

In [604]:
print_top_n_similar('경기도 성남시 분당구', topn=10, emb_rslt=emb_rslt)

[('경기도 성남시 분당구', 0.0),
 ('경기도 성남시 중원구', 0.3115725510763928),
 ('경기도 성남시 수정구', 0.3215208121966362),
 ('경기도 용인시 수지구', 0.35598373786239224),
 ('경기도 의왕시', 0.6206607111626793),
 ('경기도 용인시 기흥구', 0.6246114448718667),
 ('경기도 과천시', 0.6270986873771853),
 ('경기도 안양시 동안구', 0.7132537258794605),
 ('경기도 광주시', 0.7581819952552701),
 ('서울특별시 도봉구', 0.7634899771925955)]

In [605]:
emb_rslt = pd.read_csv("embedding+도로명.csv",index_col=0)

In [606]:
print_top_n_similar('부산광역시 사하구 낙동대로', topn=10, emb_rslt=emb_rslt)

[('부산광역시 사하구 낙동대로', 0.0),
 ('부산광역시 사하구 장평로359번길', 0.04895971827461498),
 ('부산광역시 사하구 장평로377번길', 0.06278974459860365),
 ('부산광역시 사하구 승학로165번길', 0.06626033571427359),
 ('부산광역시 사하구 마하로12번길', 0.06890229982233942),
 ('부산광역시 사하구 장평로449번길', 0.0691266838353769),
 ('부산광역시 사하구 승학로161번안길', 0.07091290058029798),
 ('부산광역시 사하구 낙동대로302번길', 0.07133906846377325),
 ('부산광역시 사하구 낙동대로327번길', 0.07636399391106563),
 ('부산광역시 사하구 낙동대로309번길', 0.07855167134119001)]

In [607]:
print_top_n_similar('대전광역시 유성구 엑스포로', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 엑스포로', 0.0),
 ('대전광역시 유성구 엑스포로295번길', 0.07806320583286541),
 ('대전광역시 유성구 유성대로1707번길', 0.08548622590395505),
 ('대전광역시 유성구 전민로26번길', 0.08549530498462488),
 ('대전광역시 유성구 전민로', 0.08790896363717882),
 ('대전광역시 유성구 전민로30번길', 0.08812414836119387),
 ('대전광역시 유성구 유성대로1689번길', 0.08886309428961237),
 ('대전광역시 유성구 전민로10번길', 0.09171062009129216),
 ('대전광역시 유성구 전민로14번길', 0.09548472852302803),
 ('대전광역시 유성구 유성대로1679번길', 0.09580267307869506)]

In [608]:
emb_rslt = pd.read_csv("embedding+행정동명전체.csv",index_col=0)

In [609]:
print_top_n_similar('대전광역시 유성구 신성동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 신성동 (행정동)', 0.0),
 ('대전광역시 유성구 온천2동 (행정동)', 0.26688485146615537),
 ('대전광역시 유성구 관평동 (행정동)', 0.30515002071554165),
 ('대전광역시 유성구 전민동 (행정동)', 0.33787784822364975),
 ('대전광역시 유성구 구즉동 (행정동)', 0.43106655916270953),
 ('대전광역시 서구 만년동 (행정동)', 0.4724173418776816),
 ('대전광역시 유성구 노은2동 (행정동)', 0.5064805687841769),
 ('대전광역시 유성구 온천1동 (행정동)', 0.5269581400724686),
 ('대전광역시 유성구 노은1동 (행정동)', 0.5337740172013945),
 ('대전광역시 서구 관저1동 (행정동)', 0.5757627109535708)]

In [610]:
print_top_n_similar('경기도 성남시 수정구 신흥1동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('경기도 성남시 수정구 신흥1동 (행정동)', 0.0),
 ('경기도 성남시 수정구 신흥3동 (행정동)', 0.02063965575897908),
 ('경기도 성남시 수정구 신흥2동 (행정동)', 0.0880720668866164),
 ('경기도 성남시 수정구 수진1동 (행정동)', 0.09239236542530394),
 ('경기도 성남시 중원구 중앙동 (행정동)', 0.11568168101741116),
 ('경기도 성남시 수정구 수진2동 (행정동)', 0.15722611245528573),
 ('경기도 성남시 수정구 태평4동 (행정동)', 0.1857241066619898),
 ('경기도 성남시 중원구 성남동 (행정동)', 0.18585406258492831),
 ('경기도 성남시 수정구 태평3동 (행정동)', 0.20197994944596176),
 ('경기도 성남시 수정구 태평2동 (행정동)', 0.21881155329892354)]

In [611]:
emb_rslt = pd.read_csv("embedding+법정동명전체.csv",index_col=0)

In [612]:
print_top_n_similar('서울특별시 강남구 역삼동', topn=10, emb_rslt=emb_rslt)

[('서울특별시 강남구 역삼동', 0.0),
 ('서울특별시 강남구 도곡동', 0.1981647435539199),
 ('서울특별시 강남구 삼성동', 0.26407542458781974),
 ('서울특별시 강남구 논현동', 0.2833330731811109),
 ('서울특별시 송파구 신천동', 0.32387320830906496),
 ('서울특별시 서초구 서초동', 0.32876551115694186),
 ('서울특별시 서초구 잠원동', 0.33769702649161615),
 ('서울특별시 강남구 압구정동', 0.3415827912791295),
 ('서울특별시 서초구 반포동', 0.3556472308391262),
 ('서울특별시 송파구 잠실동', 0.3569349300278709)]

In [613]:
print_top_n_similar('대전광역시 유성구 구성동', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 구성동', 0.0),
 ('대전광역시 유성구 가정동', 0.06859540545991882),
 ('대전광역시 유성구 방현동', 0.09938082587063535),
 ('대전광역시 유성구 장동', 0.12120977426170043),
 ('대전광역시 유성구 화암동', 0.13240107673865809),
 ('대전광역시 유성구 도룡동', 0.13462485188043716),
 ('대전광역시 유성구 덕진동', 0.14327278281772204),
 ('대전광역시 유성구 어은동', 0.23005423951603843),
 ('대전광역시 유성구 추목동', 0.2765725215047087),
 ('대전광역시 유성구 전민동', 0.308614964673085)]

In [614]:
print_top_n_similar('경기도 성남시 분당구 정자동', topn=10, emb_rslt=emb_rslt)

[('경기도 성남시 분당구 정자동', 0.0),
 ('경기도 성남시 분당구 수내동', 0.19672788533236252),
 ('경기도 수원시 장안구 정자동', 0.2927821461759944),
 ('경기도 성남시 분당구 금곡동', 0.3400240845443945),
 ('서울특별시 구로구 고척동', 0.46075962938366544),
 ('인천광역시 서구 원당동', 0.4680508594487115),
 ('경기도 성남시 분당구 구미동', 0.47176049928844843),
 ('경기도 안양시 동안구 평촌동', 0.47608929804656386),
 ('경기도 수원시 장안구 송죽동', 0.4943814605559599),
 ('경기도 용인시 수지구 동천동', 0.4947552118958142)]

In [615]:
emb_rslt = pd.read_csv("embedding+상호지점명.csv",index_col=0)

In [616]:
print_top_n_similar('페리카나 용전점', topn=10, emb_rslt=emb_rslt)

[('페리카나 용전점', 0.0),
 ('주섬주섬 용전점', 0.03639719518703444),
 ('투마리치킨 한남대점', 0.04926472502880175),
 ('CU대연원룸점 대연원룸점', 0.5144963220875947),
 ('레전드스크린야구존 연향점', 0.5229589316659196),
 ('아이리스PCPOP 순천연향점', 0.5305915422309713),
 ('생돼지자갈구이 순천점', 0.5312984036745847),
 ('엉터리생고기 순천연향점', 0.5315695236127725),
 ('동아리꼬치구이 행복점', 0.5329637379931167),
 ('동대문엽기떡볶이 순천연향점', 0.5339170512922546)]

In [617]:
print_top_n_similar('진천토종순대 청라점', topn=10, emb_rslt=emb_rslt)

[('진천토종순대 청라점', 0.0),
 ('용구비어 인천청라점', 0.001308462252987419),
 ('블레스롤 인천청라점', 0.002105774743321853),
 ('신복관 청라점', 0.0035825318364400838),
 ('쌔카스팀세차 인천청라점', 0.0049071292268458855),
 ('60계치킨 인천청라지점', 0.005427344905596931),
 ('점프셈교실 청라클라스학습관리센터', 0.006356643920865768),
 ('고씨네 샤브앤전골', 0.007253777834811959),
 ('숙이네조개전골 인천청라점', 0.008558619660631539),
 ('홈돈 청라점', 0.00951537862532259)]

In [618]:
print_top_n_similar('햇잎갈비 대전어은동카이스트점', topn=10, emb_rslt=emb_rslt)

[('햇잎갈비 대전어은동카이스트점', 0.0),
 ('폼프리츠 카이스트점', 0.10547322064607523),
 ('허니빈스 어은점', 0.11460252379703473),
 ('쿠마키친 대전어은점', 0.11560565432788816),
 ('백암이네순대국밥 월평점', 0.11723888606237387),
 ('세븐일레븐 카이스트점', 0.11978045357708433),
 ('CU 어은한빛점', 0.12215796901826248),
 ('황궁쟁반짜장 카이스트점', 0.1407088582001369),
 ('도네누 월평점', 0.14134322836172386),
 ('오땅비어 월평점', 0.14796121448963762)]

In [619]:
print_top_n_similar('파스쿠찌 팔공산점', topn=10, emb_rslt=emb_rslt)

[('파스쿠찌 팔공산점', 0.0),
 ('투썸플레이스 팔공산점', 0.0002982865084376618),
 ('파스쿠찌 팔공산온천호텔점', 0.0009979167126145105),
 ('GS25팔공산점 팔공산점', 0.003078791674171022),
 ('나들목버섯집 동화사점', 0.09403827354478445),
 ('CU 팔공산만남점', 0.09589848425988856),
 ('돈토리숯불갈비 팔공산점', 0.2028092393330927),
 ('마당새 본점', 0.2137682805516048),
 ('수정식당 본점', 0.2271588781441396),
 ('핸즈커피 팔공산아너스점', 0.24291480354736628)]

In [652]:
df[df['법정동명'] == '어은동']['상호지점명'].sample(10)

138497    세븐일레븐 카이스트점
64489             NaN
321288            NaN
453491            NaN
76473             NaN
492489            NaN
500369            NaN
309514            NaN
92110             NaN
273195            NaN
Name: 상호지점명, dtype: object

In [656]:
df[df['상호명'] == '세븐일레븐'].shape

(6670, 49)

In [620]:
emb_rslt = pd.read_csv("embedding+상호명.csv",index_col=0)

In [621]:
print_top_n_similar('삼부자부대찌개', topn=10, emb_rslt=emb_rslt)

[('삼부자부대찌개', 0.0),
 ('해병이네', 0.004218627467283631),
 ('삼부자식당', 0.06307634934076811),
 ('기시맹', 0.08627871721139269),
 ('띵호와.서울뚝배기.설악칡냉면', 0.08940888211729558),
 ('달빛구름', 0.08947957361331094),
 ('111-7밀', 0.09016612095116328),
 ('111-7meal', 0.09036755667397378),
 ('우꼬돈찌', 0.0934539017998296),
 ('큰나무사천훠궈양꼬치', 0.09456590487535654)]

In [622]:
print_top_n_similar('두메숯불바베큐', topn=10, emb_rslt=emb_rslt)

[('두메숯불바베큐', 0.0),
 ('요시다', 0.0009826457428858681),
 ('올리브가든수제파이전문점', 0.0026535577409834143),
 ('짜카밥', 0.0035928046234886546),
 ('장돌뱅이족발보쌈전문점', 0.003764339876830336),
 ('뿌구리매운갈비찜', 0.0038263915272146454),
 ('플라스틱플랜트', 0.004524935189668455),
 ('기시맹', 0.004921483119968095),
 ('베리신주꾸', 0.006411985140845067),
 ('에프터코멧', 0.012600560435873986)]

In [623]:
print_top_n_similar('콜마르브레드', topn=10, emb_rslt=emb_rslt)

[('콜마르브레드', 0.0),
 ('민스바이젠', 0.03065198344575648),
 ('어은통신', 0.051247005460349215),
 ('어흥이동통신', 0.05153325720487156),
 ('어은열쇠', 0.05753352686690105),
 ('예원입시학원', 0.12947552982738986),
 ('오네뜨', 0.2522676447360488),
 ('올마이티치즈버거', 0.2954213441628568),
 ('왔다삼겹살박준수', 0.30006478495554323),
 ('왔다!삼겹살1', 0.3004046402368076)]

In [624]:
emb_rslt = pd.read_csv("embedding+도로명주소.csv",index_col=0)

In [625]:
print_top_n_similar('서울특별시 강남구 선릉로93길 6', topn=10, emb_rslt=emb_rslt)

[('서울특별시 강남구 선릉로93길 6', 0.0),
 ('서울특별시 중구 퇴계로26길 7-10', 0.5360846292601509),
 ('충청북도 청주시 상당구 월평로184번길 54', 0.5590540922763272),
 ('경기도 광주시 순암로 367-15', 0.568711864582204),
 ('대전광역시 동구 계족로 94', 0.5741367663440762),
 ('경상남도 진주시 남강로659번길 15', 0.5845793655526974),
 ('경기도 고양시 덕양구 화신로260번길 74', 0.5928512346541015),
 ('경기도 이천시 중리천로 77', 0.5970678573756292),
 ('대구광역시 달서구 성서서로69길 29', 0.6086937976557598),
 ('전라남도 순천시 해룡면 조례못등길 98', 0.617102129011039)]

In [626]:
print_top_n_similar('대전광역시 유성구 엑스포로123번길 65-38', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 엑스포로123번길 65-38', 0.0),
 ('전라북도 전주시 완산구 화산천변6길 7', 0.5103605600829955),
 ('전라북도 전주시 완산구 새터로 52-14', 0.5149250909948245),
 ('강원도 강릉시 성덕포남로 104', 0.5367215593783916),
 ('전라북도 전주시 완산구 화산천변6길 5', 0.5384242803531858),
 ('서울특별시 강남구 삼성로64길 51', 0.5434488522430065),
 ('대구광역시 달서구 달구벌대로259길 32', 0.5499070356133464),
 ('부산광역시 동래구 충렬대로459번길 37', 0.5500985062596828),
 ('대구광역시 수성구 달구벌대로637길 22-3', 0.5553342544154769),
 ('경상남도 진주시 초북로 36', 0.5581164283196136)]

In [627]:
emb_rslt = pd.read_csv("embedding+지번주소.csv",index_col=0)

In [628]:
print_top_n_similar('대전광역시 유성구 도룡동 4-30', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 도룡동 4-30', 0.0),
 ('대전광역시 유성구 도룡동 4-8', 0.25233591418429613),
 ('충청남도 보령시 신흑동 1901', 0.28005585232726826),
 ('전라남도 여수시 돌산읍 금성리 847-6', 0.2840058804739206),
 ('부산광역시 해운대구 우동 618-1', 0.28492164151876165),
 ('경기도 가평군 조종면 운악리 산163', 0.29332243273867276),
 ('강원도 속초시 조양동 1466-1', 0.2938589914147913),
 ('제주특별자치도 제주시 한림읍 금능리 1474-11', 0.295693804819287),
 ('전라남도 여수시 돌산읍 우두리 1033-8', 0.29571658295081993),
 ('서울특별시 서대문구 창천동 57-17', 0.2966391211934274)]

In [629]:
print_top_n_similar('대전광역시 유성구 궁동 424-2', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 궁동 424-2', 0.0),
 ('전라북도 익산시 신동 796-1', 0.15233402865471868),
 ('대전광역시 유성구 봉명동 611-3', 0.2573150185191162),
 ('대전광역시 유성구 궁동 432-9', 0.259373598763757),
 ('대전광역시 유성구 궁동 404-6', 0.26055762994406373),
 ('대전광역시 유성구 궁동 418-8', 0.26139511377526214),
 ('대전광역시 유성구 궁동 425-12', 0.262656097953779),
 ('대전광역시 유성구 궁동 424-6', 0.2632454698442258),
 ('대전광역시 유성구 궁동 498', 0.2635579586083725),
 ('대전광역시 유성구 봉명동 645-4', 0.26482127869564864)]

In [630]:
emb_rslt = pd.read_csv("embedding+상권업종소분류명.csv",index_col=0)

In [631]:
emb_rslt

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,...,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
호프/맥주,-0.065,-0.022,-0.064,-0.012,-0.036,0.104,-0.088,-0.137,0.039,0.039,-0.054,-0.027,0.056,0.100,-0.044,-0.047,-0.008,0.055,-0.043,...,-0.097,0.101,-0.139,-0.002,-0.075,0.057,-0.037,0.111,-0.017,-0.007,0.054,-0.083,0.023,-0.033,-0.092,0.090,0.109,-0.081,-0.002
커피전문점/카페/다방,-0.033,0.008,-0.038,-0.007,-0.059,0.054,-0.080,-0.105,0.066,0.032,-0.069,-0.047,0.076,0.103,-0.030,-0.029,0.008,0.061,-0.013,...,-0.090,0.116,-0.127,-0.013,-0.025,0.063,-0.027,0.117,-0.005,-0.023,0.043,-0.063,0.023,-0.048,-0.084,0.074,0.087,-0.075,0.002
토스트전문,-0.013,-0.012,-0.046,0.010,-0.048,0.076,-0.048,-0.101,0.062,0.053,-0.031,-0.038,0.065,0.081,-0.031,-0.054,-0.017,0.036,-0.006,...,-0.056,0.112,-0.142,-0.040,-0.073,0.048,-0.040,0.088,0.025,-0.036,0.075,-0.049,-0.001,-0.057,-0.068,0.099,0.069,-0.077,-0.033
사우나/증기탕/온천,-0.058,0.084,-0.023,-0.036,0.023,0.033,0.034,0.003,0.082,0.098,-0.039,-0.063,0.034,-0.036,0.094,0.022,0.055,0.008,0.019,...,0.004,-0.071,0.256,0.156,0.047,-0.027,-0.009,-0.227,0.022,0.012,-0.042,-0.131,0.171,-0.055,0.063,-0.095,0.053,-0.000,0.036
음식점-초밥전문,-0.058,0.034,-0.026,-0.040,-0.057,0.066,-0.094,-0.124,0.029,0.024,-0.099,-0.051,0.069,0.117,-0.062,-0.056,-0.003,0.048,-0.007,...,-0.092,0.122,-0.140,0.002,-0.037,0.069,-0.028,0.151,-0.028,-0.010,0.049,-0.067,0.019,-0.042,-0.112,0.069,0.111,-0.086,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
석궁용품판매,0.020,0.018,0.090,0.009,-0.014,-0.064,-0.018,-0.001,-0.014,-0.090,0.005,0.079,-0.032,0.010,-0.014,-0.019,-0.012,-0.039,-0.016,...,-0.027,-0.019,-0.037,0.001,0.038,-0.078,0.097,0.049,0.016,0.021,-0.048,0.064,-0.009,0.065,0.051,-0.024,-0.037,0.002,0.005
학원-차트,0.094,0.041,-0.023,0.060,-0.011,-0.068,0.041,0.100,-0.159,0.043,0.074,-0.020,0.005,-0.023,-0.035,-0.046,-0.016,-0.025,0.009,...,0.129,-0.025,-0.050,-0.101,-0.022,-0.054,-0.038,-0.033,0.064,-0.006,0.038,0.124,-0.076,-0.060,0.060,-0.010,-0.060,0.055,-0.054
게이트볼장,0.093,-0.091,0.071,-0.058,0.037,0.011,-0.067,0.045,-0.011,-0.102,0.002,0.011,0.022,-0.074,0.047,-0.033,0.063,0.093,0.003,...,0.041,-0.123,0.046,-0.102,-0.034,0.068,-0.056,-0.044,-0.063,0.018,-0.012,0.073,0.039,0.148,-0.048,-0.034,-0.083,-0.025,-0.190
실외골프연습장,0.062,-0.078,0.003,-0.028,0.039,-0.018,-0.068,0.130,0.050,-0.086,0.074,-0.029,0.037,-0.068,0.088,-0.034,0.027,0.068,0.028,...,0.073,-0.172,0.058,-0.139,-0.051,-0.037,-0.044,-0.069,-0.036,0.005,-0.018,0.094,0.070,0.029,-0.047,-0.047,-0.132,0.032,-0.106


In [632]:
print_top_n_similar('실외골프연습장', topn=10, emb_rslt=emb_rslt)

[('실외골프연습장', 0.0),
 ('체육관', 0.10426074918846728),
 ('쿵후도장', 0.1046527161806029),
 ('복싱도장', 0.1082697607545915),
 ('기타실내운동시설', 0.10833469970911014),
 ('스포츠센터', 0.11049489133319723),
 ('테니스장', 0.11052423644266829),
 ('기타경기후원업/경기단체', 0.11059689532911998),
 ('실내골프연습장', 0.11074257556060785),
 ('킥복싱', 0.11088417751909119)]

In [633]:
print_top_n_similar('족발/보쌈전문', topn=10, emb_rslt=emb_rslt)

[('족발/보쌈전문', 0.0),
 ('갈비/삼겹살', 0.0018667048413583132),
 ('설렁탕집', 0.002654282554316545),
 ('쌈밥전문', 0.0031534440998020052),
 ('기사식당', 0.0036656756913956423),
 ('냉면집', 0.004078516098055762),
 ('돌솥/비빕밥전문점', 0.004181284830792942),
 ('한정식전문', 0.004356089063629942),
 ('부대찌개/섞어찌개', 0.00443824136057136),
 ('한식/백반/한정식', 0.005451930880259859)]

In [634]:
print_top_n_similar('피아노/바이올린/기타', topn=10, emb_rslt=emb_rslt)

[('피아노/바이올린/기타', 0.0),
 ('서예/서화/미술', 0.00016853727498922222),
 ('합기도장', 0.0005442661655647996),
 ('어린이집', 0.000611113063410107),
 ('검도도장', 0.0006629770955132175),
 ('학원(종합)', 0.0007274648191920585),
 ('실내놀이터-유아', 0.0007556705473990011),
 ('학원-웅변', 0.0007636609767125435),
 ('학원-제과기술', 0.0007698276395181614),
 ('학습지보급', 0.0007854504513449267)]

In [635]:
print_top_n_similar('당구장', topn=10, emb_rslt=emb_rslt)

[('당구장', 0.0),
 ('전자오락실', 0.0013520106466263782),
 ('무도장', 0.0013555893406873576),
 ('비디오감상실', 0.0014363968039283748),
 ('단식원', 0.0014369438351939934),
 ('나이트클럽', 0.0014963156535887645),
 ('연극/음악/예술관련기타', 0.0015067287607921642),
 ('실내야구연습장', 0.0015457576488356883),
 ('지압원', 0.0015508212948635025),
 ('극장식유흥주점', 0.0015566608162713713)]

In [636]:
df[df['법정동명']=='궁동']['상권업종소분류명'].sample(10)

26860            수퍼마켓
426682          여성미용실
149645      한식/백반/한정식
492959    커피전문점/카페/다방
266650        자동차부품판매
375030        해장국/감자탕
382300          부동산중개
354984            편의점
386639    커피전문점/카페/다방
15632             렌트카
Name: 상권업종소분류명, dtype: object

In [670]:
df[df['상권업종대분류명']=='의료']['상호명'].sample(10)

336748    이보코리아치과기공소
137953       구미예치과병원
775019      에스앤케이본치과
14294           아리약국
196178         BNE치과
378145       정극모치과의원
229872          해동병원
506239        한수동물병원
467507         신례원의원
466999       한빛정신과의원
Name: 상호명, dtype: object